## Task 3
В этом упражнении ты создашь так называемую витрину данных. Она представляет собой таблицу, 
которую можно использовать для аналитических целей. Обычно она создается путем объединения нескольких отдельных таблиц. 
В этом упражнении мы будем собирать различные данные о наших пользователях: когда они сделали свои первые коммиты, 
когда они впервые посетили ленту новостей и т. д. Это поможет позднее выполнить анализ данных.

Что тебе нужно сделать в этом упражнении (ознакомься с полным описанием задания):

## 1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.

In [ ]:
! pip install sqlalchemy

In [47]:
import pandas as pd
import numpy as np
import sqlite3
from pandas import DataFrame

In [48]:
conn = sqlite3.connect("../data/checking-logs.sqlite")

## 2. Создай новую таблицу `datamart` в базе данных, объединив таблицы `pageviews` и `checker` с помощью только **одного** запроса.
    - Таблица должна содержать следующие столбцы: `uid`, `labname`, `first_commit_ts`, `first_view_ts`.
    - `first_commit_ts` — это просто новое имя для столбца `timestamp` из таблицы `checker`; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
    - `first_view_ts` — первое посещение пользователем из таблицы `pageviews`, метка времени посещения пользователем ленты новостей.
    - По-прежнему нужно использовать фильтр `status = 'ready'`.
    - По-прежнему нужно использовать фильтр `numTrials = 1`.
    - Имена лабораторных заданий по-прежнему должны быть из следующего списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`.
    - Таблица должна содержать только пользователей (`uid` с `user_*`), а не администраторов.
    - `first_commit_ts` и `first_view_ts` должны быть распарсены как `datetime64[ns]`.

In [49]:
datamart = pd.io.sql.read_sql("""SELECT c.uid, c.labname, c.timestamp AS first_commit_ts,
                            MIN(p.datetime) AS first_view_ts
                            FROM checker AS c
                            LEFT JOIN pageviews AS p
                            ON c.uid=p.uid
                            WHERE c.status='ready' 
                            AND c.numTrials=1 
                            AND c.labname IN ('laba04','laba04s','laba05','laba06','laba06s','project1')
                            AND c.uid LIKE 'user_%'
                            GROUP BY c.timestamp, c.uid;""", 
                            conn, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.to_sql('datamart', conn, if_exists='replace')                                                  

140

In [50]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## 3. Используя методы библиотеки Pandas, создай два датафрейма: `test` и `control`.
    - `test` должен включать пользователей, у которых имеются значения в `first_view_ts`.
    - `control` должен включать пользователей, у которых отсутствуют значения в `first_view_ts`.
    - Замени пропущенные значения в `control` средним значением `first_view_ts` пользователей из `test` (оно пригодится нам для анализа в будущем).
    - Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [51]:
test = datamart.dropna().copy()
control = datamart[datamart.first_view_ts.isna()].copy()
control.first_view_ts = test.first_view_ts.mean()
test.to_sql('test', conn, if_exists='replace')
control.to_sql('control', conn, if_exists='replace')

81

In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [53]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


## 4. Закрой соединение.

In [56]:
conn.close()

In [59]:
conn = sqlite3.connect("../data/checking-logs.sqlite")

In [60]:
test = pd.io.sql.read_sql('SELECT * FROM test', conn)

In [61]:
control = pd.io.sql.read_sql('SELECT * FROM control', conn)